In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Adrenocortical_Cancer/GSE49278'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression profiling by array of 44 adrenocortical carcinomas"
!Series_summary	"Gene expression profiles of adrenocortical carcinomas were analyzed using Affymetrix Human Gene 2.0 ST Array to identify homogeneous molecular subgroups"
!Series_overall_design	"Gene expression profiles of 44 adrenocortical carcinomas were hybridized using Affymetrix Human Gene 2.0 ST Array"
Sample Characteristics Dictionary:
{0: ['age (years): 70', 'age (years): 26', 'age (years): 53', 'age (years): 73', 'age (years): 15', 'age (years): 51', 'age (years): 63', 'age (years): 29', 'age (years): 79', 'age (years): 45', 'age (years): 43', 'age (years): 41', 'age (years): 37', 'age (years): 81', 'age (years): 68', 'age (years): 42', 'age (years): 59', 'age (years): 39', 'age (years): 25', 'age (years): 36', 'age (years): 24', 'age (years): 49', 'age (years): 75', 'age (years): 48', 'age (years): 54', 'age (years): 28', 'age (years): 40', 'age (years): 44', 'age (years): 52

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Corrected variable name
Sample_Characteristics_Dictionary = {
    0: ['age (years): 70', 'age (years): 26', 'age (years): 53', 'age (years): 73', 'age (years): 15', 'age (years): 51', 'age (years): 63', 'age (years): 29', 'age (years): 79', 'age (years): 45', 'age (years): 43', 'age (years): 41', 'age (years): 37', 'age (years): 81', 'age (years): 68', 'age (years): 42', 'age (years): 59', 'age (years): 39', 'age (years): 25', 'age (years): 36', 'age (years): 24', 'age (years): 49', 'age (years): 75', 'age (years): 48', 'age (years): 54', 'age (years): 28', 'age (years): 40', 'age (years): 44', 'age (years): 52', 'age (years): 30'],
    1: ['gender: F', 'gender: M'],
    2: ['cell type: Adrenocortical carcinoma']
}

# Determine if gene expression data is available
if any(keyword in "!Series_summary" for keyword in ["Gene expression profiles", "Expression profiling by array"]):
    is_gene_available = True

# Identify if data is available for each variable and set the necessary rows
if any("age" in characteristic for characteristic in Sample_Characteristics_Dictionary[0]):
    age_row = 0

if any("gender" in characteristic for characteristic in Sample_Characteristics_Dictionary[1]):
    gender_row = 1

if "Adrenocortical carcinoma" in Sample_Characteristics_Dictionary[2][0]:
    trait_row = 2

# Define conversion functions for age, gender, and trait

# Age conversion function (continuous type)
def convert_age(value):
    try:
        return float(value.split(': ')[1])
    except:
        return None

# Gender conversion function (binary type: female -> 0, male -> 1)
def convert_gender(value):
    try:
        gender = value.split(': ')[1].strip().upper()
        return 0 if gender == 'F' else 1 if gender == 'M' else None
    except:
        return None

# Trait conversion function (binary type: Adrenocortical carcinoma -> 1, otherwise -> None since it's a binary condition based on presence or absence)
def convert_trait(value):
    return 1 if "Adrenocortical carcinoma" in value else None

# Save cohort information
save_cohort_info('GSE49278', './preprocessed/Adrenocortical_Cancer/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(
        clinical_data, 'Adrenocortical_Cancer', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender
    )
    csv_path = './preprocessed/Adrenocortical_Cancer/trait_data/GSE49278.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM1196511': [1.0, 70.0, 0.0], 'GSM1196512': [1.0, 26.0, 0.0], 'GSM1196513': [1.0, 53.0, 0.0], 'GSM1196514': [1.0, 73.0, 1.0], 'GSM1196515': [1.0, 15.0, 0.0], 'GSM1196516': [1.0, 51.0, 0.0], 'GSM1196517': [1.0, 63.0, 1.0], 'GSM1196518': [1.0, 26.0, 0.0], 'GSM1196519': [1.0, 29.0, 1.0], 'GSM1196520': [1.0, 79.0, 0.0], 'GSM1196521': [1.0, 45.0, 0.0], 'GSM1196522': [1.0, 43.0, 0.0], 'GSM1196523': [1.0, 53.0, 0.0], 'GSM1196524': [1.0, 45.0, 0.0], 'GSM1196525': [1.0, 41.0, 0.0], 'GSM1196526': [1.0, 37.0, 0.0], 'GSM1196527': [1.0, 81.0, 0.0], 'GSM1196528': [1.0, 68.0, 1.0], 'GSM1196529': [1.0, 42.0, 0.0], 'GSM1196530': [1.0, 59.0, 0.0], 'GSM1196531': [1.0, 39.0, 0.0], 'GSM1196532': [1.0, 25.0, 0.0], 'GSM1196533': [1.0, 41.0, 0.0], 'GSM1196534': [1.0, 36.0, 0.0], 'GSM1196535': [1.0, 24.0, 0.0], 'GSM1196536': [1.0, 49.0, 0.0], 'GSM1196537': [1.0, 75.0, 0.0], 'GSM1196538': [1.0, 37.0, 0.0], 'GSM1196539': [1.0, 26.0, 0.0], 'GSM1196540': [1.0, 48.0, 0.0], 'GSM1196541': [1.0, 15.0, 0.0], 'GSM119